# AWS



First we install `awscli` which is aws API (follow [Link](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to install).

```zsh
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
```

```zsh
which aws
aws --version
```

Commands that expect aws credential will first look for KEY_ID and SECRET_ACCESS env variables, then local/global `.credentials` set up by  
```zsh
aws configure  # configures the credentials
aws s3 ls      # list all S3 buckets
aws sts get-caller-identity  # gets identity of a user
```


## Set up new conda environment

SageMaker comes with many built-in [Images](https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-images.html) that have many preloaded packages. Each image can support different Jupyter kernels (or equivalent conda environments).

First access Image terminal directly (navigate to the Launcher (click on "Amazon SageMaker Studio in the top left corner) and "Open Image terminal")


```zsh
conda env list
```
will print `opt/conda` only.

I had to install pip (for some reason it is not preinstalled):
```zsh
apt-get update
apt-get install python-pip
```

Let's create a new environment:
```zsh
conda create -n  python=3.9
conda activate new_env
conda install jupyter
```

Let's also add this environment as a Jupyter kernel:
```
jupyter kernelspec list
```
```
ipython kernel install --name new_env --user
```
Now our custom environment is showing up.

Let's install required packages in the `new_env` env:

```zsh
pip install streamlit pandas polars
pip install torch torchvision torchaudio
pip install jupyter methodtools pytorch-lightning scikit-learn colorama libtmux onnxruntime openpyxl xlsxwriter matplotlib pulp
```
most of these are already preinstalled.

That's it. Our kernel is now ready to be used.

There are [more](https://aws.amazon.com/blogs/machine-learning/four-approaches-to-manage-python-packages-in-amazon-sagemaker-studio-notebooks/) ways this can be done and I dabbled in [Lifecycle Configuration](#optional-set-up-lifecycle-configuration-not-functional-yet) but haven't had success.

## Clone GitHub repository

Best way to do this is via terminal (SageMaker has GitHub tab option but couldn't make it run). We'll show here one of the easy ways to do this is via Personal Access Token (there is ssh-key option as well). First, In Github, navigate to [Personal Access Tokens](https://github.com/settings/tokens), then in SageMaker Studio terminal clone the repo:
```
git clone <repo-URL>
```
**when asked for password enter the token that you just created!**

To set up global git credentials, edit and run following lines:
```zsh
git config --global user.email "you@example.com"
git config --global user.name "Your Name"
```
that way all the git changes will have your credentials.

## To run SageMaker in VS-Code web UI

SageMaker Studio is the AWSs' UI. If you prefer the VSCode look then you need to install somewhat awkwardly named "Code Server" (see installation [instructions](https://aws.amazon.com/blogs/machine-learning/host-code-server-on-amazon-sagemaker/). You will now be able to run VS Code in a browser.

![Alt Text](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2022/10/14/Screenshot-2022-09-29-at-11.33.15-2-1024x267.png)


## S3 access

For s3 access one must set up credentials.

Accessing S3 files is not as straightforward as if having them locally since interaction has to go via AWS APIs. `boto3` and `s3fs` are two libraries that have their own APIs. I find s3fs to be better since "S3Fs is a Pythonic file interface to S3. It builds on top of botocore". Pandas is an exception and `pd.read_csv` just works. There is `s3fs-fuse` library (and apparently one with even higher performance called `goofys`, neither in pip) that kind of offer mounting s3 option, however, I ran into issues when trying to install them. Following are some examples:  

In [ ]:
import boto3
import s3fs
import pandas as pd
import os

bucket_name = 'X'
file_key = 'Y'

## Read into DataFrame

In [ ]:
df = pd.read_csv(f's3://{bucket_name}/{file_key}')

## s3fs

In [ ]:
s3 = s3fs.S3FileSystem(anon=False)  # Set anon to True for public buckets

### ls

In [ ]:
contents = s3.ls(bucket_name)
print(contents)

### glob

In [ ]:
s3.glob(f's3://{bucket_name}/**/a*.csv')

### download

In [ ]:
# Specify local directory for downloaded file
local_directory = '/tmp/'
local_file_path = os.path.join(local_directory, 'Z')

# Ensure the local directory exists
os.makedirs(local_directory, exist_ok=True)

# Download CSV file
s3.download(f's3://{bucket_name}/{file_key}', local_file_path)

# Load CSV into Pandas DataFrame
df = pd.read_csv(local_file_path)

# Clean up downloaded file
# os.remove(local_file_path)

## boto3

In [ ]:
s3_client = boto3.client('s3')  

### download

In [ ]:
# Specify local directory for downloaded file
local_directory = '/tmp/'
local_file_path = os.path.join(local_directory, 'Z')

# Ensure the local directory exists
os.makedirs(local_directory, exist_ok=True)

# Download CSV file
s3_client.download_file(bucket_name, file_key, local_file_path)

# Load CSV into Pandas DataFrame
df = pd.read_csv(local_file_path)

# Clean up downloaded file
os.remove(local_file_path)


### ls

In [ ]:
contents = s3_client.list_objects(Bucket=bucket_name, Prefix='X')['Contents']  
for f in contents:  
    print(f['Key'])

## Restore deleted files

If Version Control is enabled in S3 then deleted files will just have a tag `Deleted Marker` or similar. The idea is to remove these tags ([link](https://repost.aws/knowledge-center/s3-undelete-configuration)).
To do this for many files at once first create a script that will list all the files to be restored:
```zsh
echo '#!/bin/bash' > undeleteScript.sh

aws --output text s3api list-object-versions --bucket <bucket name> --prefix "Flagging Tool/" | grep -E "^DELETEMARKERS" | awk '{FS = "[\t]+"; print "aws s3api delete-object --bucket <bucket name>  --key \42"$3"\42 --version-id "$5";"}' >> undeleteScript.sh
```

Then just run the
```zsh
chmod +x undeleteScript.sh
./undeleteScript.sh
```

finally remove it:
```zsh
rm -f undeleteScript.sh
```